# Notebook: Visualización de distribución por clases y construcción de una CNN
# Autor: generado automáticamente
# Objetivo: cargar 'datos_sisfall_completo.csv', mostrar distribución por clases, preparar ventanas (WINDOW=10, 6 features), definir/entrenar/guardar una CNN.


In [ ]:
%pip install -q pandas numpy matplotlib seaborn scikit-learn tensorflow joblib
# Si ya tienes estas librerías puedes comentar la línea anterior.


In [ ]:
# 1) Importar librerías y mostrar versiones
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

print('pandas', pd.__version__)
print('numpy', np.__version__)
print('tensorflow', tf.__version__)

# GPU/Device info (si aplica)
try:
    devices = tf.config.list_physical_devices()
    print('Devices:', devices)
except Exception as e:
    print('No se pudo listar dispositivos:', e)

sns.set(style='whitegrid')


In [ ]:
# 2) Cargar y explorar dataset
csv_path = Path('Codigos raspberry') / 'datos_sisfall_completo.csv'
print('Buscando CSV en:', csv_path)

if not csv_path.exists():
    print('Archivo no encontrado:', csv_path)
else:
    df = pd.read_csv(csv_path)
    display(df.head())
    print('\nInfo:')
    print(df.info())
    if 'state' in df.columns:
        counts = df['state'].value_counts().sort_index()
        print('\nDistribución por estado (conteos):')
        print(counts)
        pct = (counts / counts.sum() * 100).round(2)
        print('\nPorcentajes:')
        print(pct)
        fig, ax = plt.subplots(1,2, figsize=(12,4))
        sns.barplot(x=counts.index.astype(str), y=counts.values, ax=ax[0])
        ax[0].set_title('Distribución por clase (conteos)')
        ax[0].set_xlabel('state')
        ax[0].set_ylabel('conteos')
        ax[1].pie(counts.values, labels=counts.index.astype(str), autopct='%1.1f%%')
        ax[1].set_title('Distribución por clase (porcentaje)')
        plt.show()
    else:
        print("La columna 'state' no existe en el CSV. Revisar el archivo de entrada.")


In [ ]:
# 3) Preparar ventanas deslizantes (WINDOW_SIZE=10) y features
FEATURES = ['ax','ay','az','gx','gy','gz']
WINDOW_SIZE = 10

assert 'df' in globals(), "Ejecuta la celda de carga de dataset primero." 

# Función para crear ventanas agrupando por columna 'file' si existe, si no crear una única secuencia
def create_windows(df, features, window_size, group_col_candidates=('file','file_name','sequence','trial')):
    groups = None
    for c in group_col_candidates:
        if c in df.columns:
            groups = [g for _, g in df.groupby(c)]
            print(f'Agrupando por columna: {c}')
            break
    if groups is None:
        groups = [df]
        print('No hay columna de agrupación, procesando dataset como secuencia única')

    X_list = []
    y_list = []
    for g in groups:
        # asegurarnos de ordenar por índice si procede
        g = g.reset_index(drop=True)
        n = len(g)
        if n < window_size:
            continue
        for i in range(n - window_size + 1):
            win = g.loc[i:i+window_size-1, features]
            if win.isnull().any().any():
                continue
            X_list.append(win.values.astype(np.float32))
            # etiqueta: moda de 'state' en la ventana
            if 'state' in g.columns:
                mode = g.loc[i:i+window_size-1, 'state'].mode()
                label = mode.iloc[0] if len(mode)>0 else g.loc[i, 'state']
            else:
                label = 0
            y_list.append(int(label))
    X = np.array(X_list)
    y = np.array(y_list)
    return X, y

X, y = create_windows(df, FEATURES, WINDOW_SIZE)
print('X shape:', X.shape)
print('y shape:', y.shape)

if y.size>0:
    import pandas as _pd
    print('\nDistribución de etiquetas tras windowing:')
    print(_pd.Series(y).value_counts().sort_index())
    fig, ax = plt.subplots(figsize=(6,3))
    sns.countplot(x=y, ax=ax)
    ax.set_title('Etiquetas por ventana')
    plt.show()
else:
    print('No se generaron ventanas. Revisa el CSV y las columnas de features/state.')


In [ ]:
# 4) Normalizar (StandardScaler) y guardar scaler
if X.size>0:
    n_windows, w, n_features = X.shape
    X_reshaped = X.reshape(-1, n_features)
    scaler = StandardScaler()
    X_scaled_flat = scaler.fit_transform(X_reshaped)
    X_scaled = X_scaled_flat.reshape(n_windows, w, n_features)
    # guardar scaler
    scaler_path = Path('Codigos raspberry') / 'scaler.save'
    joblib.dump(scaler, scaler_path)
    print('Scaler guardado en', scaler_path)
    X = X_scaled
else:
    print('Nada que normalizar (X vacío)')


In [ ]:
# 5) Train/Test split
if X.size>0:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)
else:
    print('No hay datos para split')


In [ ]:
# 6) Definir arquitectura CNN
if X.size>0:
    n_classes = len(np.unique(y))
    input_shape = (WINDOW_SIZE, len(FEATURES))
    def build_model(input_shape, n_classes):
        model = models.Sequential()
        model.add(layers.Conv1D(32, 3, activation='relu', input_shape=input_shape))
        model.add(layers.MaxPooling1D(2))
        model.add(layers.Conv1D(64, 3, activation='relu'))
        model.add(layers.MaxPooling1D(2))
        model.add(layers.Flatten())
        model.add(layers.Dense(64, activation='relu'))
        model.add(layers.Dropout(0.3))
        model.add(layers.Dense(n_classes, activation='softmax'))
        return model

    model = build_model(input_shape, n_classes)
    model.summary()
else:
    print('No hay datos para construir el modelo')


In [ ]:
# 7) Compilar y entrenar (o cargar si existe el modelo)
model_path = Path('Codigos raspberry') / 'modelo_cnn_imu.h5'

if X.size>0:
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    if model_path.exists():
        print('Cargando modelo existente:', model_path)
        model = keras.models.load_model(model_path)
    else:
        callbacks = [keras.callbacks.ModelCheckpoint(model_path, save_best_only=True, monitor='val_loss')]
        history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1, callbacks=callbacks)
        # plot history
        fig, ax = plt.subplots(1,2, figsize=(12,4))
        ax[0].plot(history.history['loss'], label='train_loss')
        ax[0].plot(history.history['val_loss'], label='val_loss')
        ax[0].legend(); ax[0].set_title('Loss')
        ax[1].plot(history.history['accuracy'], label='train_acc')
        ax[1].plot(history.history['val_accuracy'], label='val_acc')
        ax[1].legend(); ax[1].set_title('Accuracy')
        plt.show()
else:
    print('No se puede entrenar: X vacío')


In [ ]:
# 8) Evaluación en test set
if X.size>0:
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print(f'Test loss: {loss:.4f}  Test acc: {acc:.4f}')
    y_pred_probs = model.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)
    print('\nClassification report:')
    print(classification_report(y_test, y_pred))
    cm = confusion_matrix(y_test, y_pred)
    fig, ax = plt.subplots(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
    ax.set_title('Matriz de confusión')
    ax.set_xlabel('Predicted'); ax.set_ylabel('True')
    plt.show()
else:
    print('No hay datos para evaluar')


In [ ]:
# 9) Prueba de predicción sobre una ventana de test
if X.size>0:
    LABELS = None
    try:
        # si las etiquetas son 4/5 etc., permitir mapear manualmente
        LABELS = ['pararse','sentarse','caminar','caerse','quieto'][:len(np.unique(y))]
    except Exception:
        LABELS = [str(i) for i in sorted(np.unique(y))]

    idx = 0
    sample = X_test[idx:idx+1]
    probs = model.predict(sample)[0]
    pred = np.argmax(probs)
    print('Predicted:', pred, 'label:', LABELS[pred] if pred < len(LABELS) else pred)
    print('Probabilidades:', probs)
else:
    print('No hay muestra para predecir')


# 10) Notas finales y siguientes pasos

# Siguientes pasos recomendados:
# - Aumentar epochs y ajustar batch_size si tienes suficiente CPU/RAM.
# - Guardar y reutilizar el scaler en el cliente real (`conexcion.py`) para normalizar en producción.
# - Si la clase 'quieto' u otras están desbalanceadas, considerar técnicas de balanceo (SMOTE, oversampling o ponderar la pérdida).
# - Validar el modelo en datos reales del Arduino (usando el pipeline de BLE -> `conexcion.py`).

print('Notebook listo. Ejecuta celdas en orden: instalar dependencias -> carga CSV -> crear ventanas -> entrenar/evaluar.')
